# <h1 style="color: purple;">Step 1 - Scraping</h1>

### NASA Mars News

In [1]:
# Import Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Init Browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Scrape the NASA Mars News Site here (https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest)
# and collect the LATEST News TITLE and PARAGRAPH Text. Assign the text to variables that you can reference later.
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Get the list of news 
ul_list_news = soup.find('ul', class_='item_list')
# Get the last news
li_last_news = ul_list_news.find_all('li')[0]
# Get the link for the last news.
link_latest_news = li_last_news.find('a')['href']
print(link_latest_news)

/news/8506/whats-mars-solar-conjunction-and-why-does-it-matter/


In [4]:
# Go to the news page
url = url.replace("/news","",1) + link_latest_news
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Get the title
last_title = soup.find('h1', class_='article_title')
last_title = last_title.text

# Get the paragraph list
lst_p = soup.find('div', class_='wysiwyg_content')
# Get the first paragraph
last_p = lst_p.find_all('p')[0]

print(last_p.text)

The daily chatter between antennas here on Earth and those on NASA spacecraft at Mars is about to get much quieter for a few weeks.


### JPL Mars Space Images - Featured Image

In [6]:
# Visit the url for JPL Featured Space Image here ==> https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars.
# Go to the news page
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# Use splinter to navigate the site and find the image url for the current Featured 
# Mars Image and assign the url string to a variable called featured_image_url.
# Get the Image URL.
carousel_item = soup.find('article', class_="carousel_item")['style']
featured_image_url = 'https://www.jpl.nasa.gov' + carousel_item.split("'")[1]

print("The URL is {}".format(featured_image_url))

The URL is https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16729-1920x1200.jpg


### Mars Weather

In [8]:
# Visit the Mars Weather twitter account here (https://twitter.com/marswxreport?lang=en) and scrape the latest 
# Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Remove tag a from tag p to get the cleaned paragraph.
for a in soup.find_all("a", {'class':'twitter-timeline-link u-hidden'}): 
    a.decompose()

# Get the Mars Weather Text
ol = soup.find('ol', class_='stream-items js-navigable-stream')
li = ol.find_all('li')[0]
div = li.find('div', class_='js-tweet-text-container')
mars_weather = div.find('p').text + "."

print (mars_weather)

InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)
winds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)
pressure at 7.70 hPa.


### Mars Facts

In [9]:
# Visit the Mars Facts webpage here (https://space-facts.com/mars/) and use Pandas to scrape 
# the table containing facts about the planet including Diameter, Mass, etc.
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
df = tables[1]
df.rename(columns={0:'Description',1:'Value'}, inplace=True)
df.set_index('Description', inplace=True)

df.head(20)

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
# Convert the data to a HTML table string and format to use bootstrap classes.  
html_table = df.to_html(escape=False, justify="left", classes="table table-striped table-bordered table-sm")
print(html_table)

<table border="1" class="dataframe table table-striped table-bordered table-sm">
  <thead>
    <tr style="text-align: left;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos & Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody

### Mars Hemispheres

In [11]:
# Visit the USGS Astrogeology site here (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars)
# to obtain high resolution images for each of Mar's hemispheres.
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Get text links to click
items = soup.find_all("div", class_="description")
lst_text =[]
for item in items:
    lst_text.append(item.h3.text)

print(lst_text)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [12]:
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# Looping to click
hemisphere_image_urls = []
for i in range( len(lst_text) ):
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    browser.click_link_by_partial_text(lst_text[i])

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    # Get the image URL
    tmp_img_url = soup.find('img',class_="wide-image")["src"]
    img_url = "https://astrogeology.usgs.gov" + tmp_img_url

    # Get the title
    title = soup.title.text
    title = title.split("|")[0]
    title = title.replace("Enhanced","")
    title = title.strip()
    # Append the dictionary with the image url string and the hemisphere title to a list. 
    hemisphere_image_urls.append({"title":title, "img_url": img_url})     

print(hemisphere_image_urls)    

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [13]:
# Close the browser
browser.quit()